In [38]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import sys
import os
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pdb
from pyspark.sql import Row
import calendar
import numpy as np

In [40]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import year
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

In [41]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [42]:
# df=spark.read.format("csv")\
#   .option('inferSchema',"true")\
#   .option('header',"true")\
#   .option('sep',",")\
#   .load("/content/drive/MyDrive/dataset_for_stream/testing.csv")

In [43]:
# df=df.filter(df.Reported_year==2010)
from pyspark.sql.functions import col
# df.filter(col("Description").like("%Stolen%")).show()

In [44]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrime2010To2018.csv")
data.columns

<ipython-input-44-d1d5032db340>:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrime2010To2018.csv")


Index(['Report_No', 'Reported_year', 'Reported_month', 'Reported_day',
       'Reported_hour', 'Reported_minute', 'From_year', 'From_month',
       'From_day', 'From_hour', 'From_minute', 'Offense', 'IBRS',
       'Description', 'Beat', 'Address', 'City', 'Zip_Code', 'Rep_Dist',
       'Area', 'DVFlag', 'Invl_No', 'Involvement', 'Race', 'Sex', 'Age',
       'Firearm_Used_Flag'],
      dtype='object')

In [45]:
#schema
sch = "Report_No int, Reported_year int, Reported_month int, Reported_day int, Reported_hour int, Reported_minute int, From_year int, From_month int, From_day int, From_hour int, From_minute int, Offense String, IBRS String, Description String, Beat String, Address String, City String, Zip_Code String, Rep_Dist String, Area String, DVFlag String, Invl_No int, Involvement String, Race String, Sex String, Age int, Firearm_Used_Flag String"
# read streaming data from a source
df = spark.readStream \
         .option("header", True) \
         .schema(sch) \
         .csv("/content/drive/MyDrive/dataset_for_stream")

In [46]:
df

DataFrame[Report_No: int, Reported_year: int, Reported_month: int, Reported_day: int, Reported_hour: int, Reported_minute: int, From_year: int, From_month: int, From_day: int, From_hour: int, From_minute: int, Offense: string, IBRS: string, Description: string, Beat: string, Address: string, City: string, Zip_Code: string, Rep_Dist: string, Area: string, DVFlag: string, Invl_No: int, Involvement: string, Race: string, Sex: string, Age: int, Firearm_Used_Flag: string]

In [47]:
# apply some transformations
filtered_stream_df = df.filter(col("Description").like("%Stolen%"))

# start the query execution
# Write the filtered DataFrame to CSV files partitioned by Reported_year
query = filtered_stream_df.writeStream \
                         .partitionBy("Reported_year") \
                         .format("csv") \
                         .option("path", "/content/drive/MyDrive/output_folder") \
                         .option("checkpointLocation", "/path/to/checkpoint_location") \
                         .start()